# Data Engineer Challenge

En este desafío, realizo un análisis de datos sobre un conjunto de tweets relacionados con las protestas de agricultores. Utilizaremos Python y algunas herramientas como para el desarrollo como `unittest`,`memory_profiler` y `Jupyter Notebook` para llevar a cabo este análisis.



---


*Para cada una de las funciones requeridas en el desafio muestro los 3 intentos más destacados (incluida la función final definitiva) en donde voy narrando lo más detalladamente posible lo que hice para llegar a la solución final.*


---



## - Preparación del entorno

A nivel local se crea un repositorio en gitHub entorno virtual que sirven para el inicio del desarrollo.

## Instalación de los requerimientos:   

Dentro del entorno virtual o el Jupyter Notebook se instalan los paquetes requeridos desde requirements.txt

In [29]:
# Instalar los paquetes desde requirements.txt
!pip install -r https://raw.githubusercontent.com/eLgRuNgE/challenge_DE/develop/requirements.txt

Se cargan los datos desde JSON

In [30]:
import zipfile
import io
import requests

# URL del archivo ZIP
url = "https://raw.githubusercontent.com/eLgRuNgE/challenge_DE/develop/data/tweets.json.zip"

# Descargar el archivo ZIP
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Obtengo el archivo requerido
target_file_name = "farmers-protest-tweets-2021-2-4.json"
zip_file.extract(target_file_name)

# Ahora `target_file_name` está disponible en el directorio actual de Colab
test_file_path = target_file_name

# **Reto #1**

Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:

```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns:
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```

# **R#1 - Enfoque 1:** Optimización del tiempo de ejecución.

### ***- Primer intento:*** función `q1_time`

La solución más obvia a este problema es usar la clase `defaultdict` que se encuentra en el módulo `collections` de Python. Funciona de manera similar a un diccionario convencional (dict), pero con una diferencia clave: automáticamente crea valores por defecto para claves que aún no están en el diccionario. Esto significa que no es necesario preocuparse por verificar si una clave existe antes de acceder a ella o asignarle un valor.

En esta implementación también se lee el archivo linea por linea para mejorar el uso de memoria y no cargar todo el archivo json en memoria.


***Descripción detallada:***

***1.   Lectura del archivo JSON:***
*   La función comienza abriendo el archivo JSON especificado en el parámetro `file_path` en modo de lectura.
*   Luego lee todas las líneas del archivo y carga los datos en una lista de listas.
*   Cada sublista contiene la fecha del tweet y el nombre de usuario del autor del tweet.
*   Estos datos se extraen del JSON utilizando la función `json.loads(line)['date'].split('T')[0]` para obtener la fecha y `json.loads(line)['user']['username']` para obtener el nombre de usuario.

***2.   Procesamiento de los datos:***
*   Se utiliza un diccionario para almacenar los pares fecha - usuario y sus respectivos conteos de tweets.
*   Para cada tweet en el archivo, se extrae la fecha y el nombre de usuario.
*   Luego, se actualiza el contador de tweets por usuario y fecha en el diccionario.

***3.   Determinación de las fechas más comunes:***
*   Utilizando `sorted()` y `key`, se determinan las 10 fechas más comunes en el diccionario.
*   Las fechas se ordenan según el número total de tweets publicados en cada una de ellas.

***4.   Obtención del usuario más activo por fecha:***
*   Para cada una de las fechas más comunes, se determina el usuario más activo.
*   Esto se logra encontrando el usuario con el mayor número de tweets para esa fecha en particular.

***5.   Formateo de las fechas:***
*   Las fechas extraídas inicialmente están en formato de cadena de texto.
*   Se utilizan para crear objetos datetime.date utilizando la función `datetime.strptime()` para convertirlas al formato deseado.

***6.   Agrupación de las fechas y usuarios:***
*   Finalmente, se combinan las fechas formateadas y los usuarios más activos en una lista de tuplas.
*   Cada tupla contiene una fecha y el usuario más activo para esa fecha.

***7.   Retorno de resultados:***
*   La función devuelve la lista de tuplas que contiene las 10 fechas principales con los usuarios más activos para cada una de esas fechas.
*   Esta función proporciona una manera eficiente de analizar datos de tweets y obtener información sobre las fechas y usuarios más activos en la plataforma.

In [31]:
import json
from collections import defaultdict
from datetime import datetime
from typing import List, Tuple

def q1_time_attempt_1(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función analiza un archivo JSON que contiene registros de tweets y devuelve las 10 fechas
    más comunes junto con el usuario más activo para cada fecha.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los registros de tweets.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas, donde cada tupla contiene una fecha
        y el usuario más activo para esa fecha.

    Raises:
        FileNotFoundError: Si no se encuentra el archivo especificado en file_path.
    """

    # Paso 1: Lectura del archivo JSON
    dates_dict = defaultdict(lambda: defaultdict(int))

    # Paso 2: Procesamiento de los datos
    with open(file_path, 'r') as data:
        for line_data in data:
            tweet = json.loads(line_data)
            tweet_date = tweet['date'].split('T')[0]
            username = tweet['user']['username']

            # Actualización del contador de tweets por usuario y fecha
            dates_dict[tweet_date][username] += 1

    # Paso 3: Determinación de las fechas más comunes
    top_dates = sorted(dates_dict.keys(), key=lambda x: sum(dates_dict[x].values()), reverse=True)[:10]

    # Paso 4: Obtención del usuario más activo por fecha
    top_users = [max(dates_dict[date], key=dates_dict[date].get) for date in top_dates]

    # Paso 5: Formateo de las fechas
    top_dates = [datetime.strptime(date_str, "%Y-%m-%d").date() for date_str in top_dates]

    # Paso 6: Agrupación de las fechas y usuarios
    result = list(zip(top_dates, top_users))

    return result

### ***- Segundo intento:*** función `q1_time`

La función `q1_time` procesa un archivo JSON que contiene registros de tweets y devuelve las 10 fechas más comunes junto con el usuario más activo para cada fecha.

***Descripción detallada:***

***1.   Obtención de los datos de fecha y usuario:***
*   Primero se guardan los datos de fecha y nombre de usuario en una lista de listas, para poder contarlos correctamente.
*   Se abre el archivo especificado en `file_path` y se lee línea por línea.
*   Los datos se almacenan en una lista donde cada elemento es otra lista que contiene la fecha del tweet y el nombre de usuario del autor.

***2.   Determinación de las fechas más comunes:***
*   Se utiliza la clase `Counter()` junto con list comprehension para obtener las 10 fechas más comunes de todos los tweets.
*   Se cuentan las ocurrencias de cada fecha en la lista de datos y se obtienen las 10 fechas más comunes utilizando el método `most_common(10)`.

***3.   Obtención del usuario más activo por fecha:***
*   De manera similar, se obtiene el usuario que más tweeteó para cada una de las fechas más comunes.
*   Se filtran los datos para cada fecha común y se cuentan las ocurrencias de cada usuario.
*   Se selecciona el usuario con mayor cantidad de tweets para cada fecha común.

***4.   Formateo de las fechas y agrupación con los usuarios:***
*   Se utilizan las fechas obtenidas anteriormente y se las convierte al formato `datetime.date` utilizando la función `datetime.strptime()`.
*   Se combinan las fechas formateadas y los usuarios más activos en una lista de tuplas.

***5.   Retorno de resultados:***
*   La función devuelve la lista de tuplas que contiene las 10 fechas principales con los usuarios más activos para cada una de esas fechas.


In [32]:
import json
from collections import Counter
from datetime import datetime
from typing import List, Tuple

def q1_time_attempt_2(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función procesa un archivo JSON que contiene registros de tweets y devuelve las 10 fechas más comunes junto con el usuario más activo para cada fecha.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los registros de tweets.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas, donde cada tupla contiene una fecha y el usuario más activo para esa fecha.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """
    # Paso 1: Obtención de los datos de fecha y usuario
    with open(file_path, 'r') as data:
        tweets_dates_users = [[json.loads(line)['date'].split('T')[0], json.loads(line)['user']['username']] for line in data.readlines()]

    # Paso 2: Determinación de las fechas más comunes
    most_common_dates = Counter([d[0] for d in tweets_dates_users]).most_common(10)

    # Paso 3: Obtención del usuario más activo para cada fecha más común
    most_common_users = [Counter([d[1] for d in tweets_dates_users if d[0] == date[0]]).most_common(1)[0][0] for date in most_common_dates]

    # Paso 4: Formateo de las fechas y agrupación con los usuarios
    result = list(zip([datetime.strptime(date[0], "%Y-%m-%d").date() for date in most_common_dates], most_common_users))

    # Paso 5: Retorno de resultados
    return result

### ***- Intento Final:*** función `q1_time`


En este intento no se almacenan los datos directamente en una lista donde cada elemento es otra lista que contiene la fecha del tweet y el nombre de usuario del autor. Tampoco se utiliza la clase Counter para optimizar la ejecución de la función `q1_time`.

***Descripción detallada:***

***1.   Obtención de los datos de fecha y usuario:***
*   Primero se leen todas las líneas del archivo JSON especificado en el parámetro `file_path`.
*   Los datos se almacenan en una lista de listas, donde cada sublista contiene la fecha del tweet y el nombre de usuario del autor.

***2.   Determinación de las fechas más comunes:***
*   Se cuenta la frecuencia de cada fecha en la lista de datos para obtener las 10 fechas más comunes.
*   No se utiliza la clase `Counter` para optimizar el código y se realiza el conteo manualmente.

***3.   Obtención del usuario más activo por fecha:***
*   Para cada una de las fechas más comunes, se filtran los datos y se cuentan las ocurrencias de cada usuario.
*   Se selecciona el usuario con mayor cantidad de tweets para cada fecha común.

***4.   Formateo de las fechas y agrupación con los usuarios:***
*   Se utilizan las fechas obtenidas anteriormente y se las convierte al formato `datetime.date` utilizando la función `datetime.strptime()`.
*   Se combinan las fechas formateadas y los usuarios más activos en una lista de tuplas.

***5.   Retorno de resultados:***
*   La función devuelve la lista de tuplas que contiene las 10 fechas principales con los usuarios más activos para cada una de esas fechas.


In [33]:
import json
from datetime import datetime
from typing import List, Tuple

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función procesa un archivo JSON que contiene registros de tweets y devuelve las 10 fechas más comunes junto con el usuario más activo para cada fecha.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los registros de tweets.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas, donde cada tupla contiene una fecha y el usuario más activo para esa fecha.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """
    # Paso 1: Obtención de los datos de fecha y usuario
    tweets_dates_users = []
    with open(file_path, 'r') as data:
        for line in data.readlines():
            tweet = json.loads(line)
            tweet_date = tweet['date'].split('T')[0]
            username = tweet['user']['username']
            tweets_dates_users.append((tweet_date, username))

    # Paso 2: Determinación de las fechas más comunes
    date_counts = {}
    for tweet_date, _ in tweets_dates_users:
        if tweet_date in date_counts:
            date_counts[tweet_date] += 1
        else:
            date_counts[tweet_date] = 1
    most_common_dates = sorted(date_counts.items(), key=lambda x: x[1],
                               reverse=True)[:10]

    # Paso 3: Obtención del usuario más activo para cada fecha más común
    most_common_users = []
    for date, _ in most_common_dates:
        user_counts = {}
        for tweet_date, username in tweets_dates_users:
            if tweet_date == date:
                if username in user_counts:
                    user_counts[username] += 1
                else:
                    user_counts[username] = 1
        most_common_user = max(user_counts, key=user_counts.get)
        most_common_users.append(most_common_user)

    # Paso 4: Formateo de las fechas y agrupación con los usuarios
    result = [(datetime.strptime(date[0], "%Y-%m-%d").date(), user)
                for date, user in zip(most_common_dates, most_common_users)]

    # Paso 5: Retorno de resultados
    return result

## - Pruebas y Ejecución de la función `q1_time`

De acuerdo a la documentación, para medir el tiempo de ejecución se recomienda el uso de [py-spy](https://github.com/benfred/py-spy) o [Python Profilers](https://docs.python.org/3/library/profile.html) sin embargo el uso de estas herramientas para una sola función lo considero engorroso ya que solo requiero medir el tiempo que tarda en ejecutarse la funcion `q1_time`.

***1. Medición de tiempo:***

Dada la explicación anterior, para la medición del tiempo de ejecución de `q1_time` solamente utilizo la función `time` nativa de Phyton para tomar un tiempo de inicio antes de la ejecución de la función y un tiempo final una vez culmina su ejecución luego calculo la diferencia entre ambos y obtengo la medición de tiempo requerida.

***2. Ejecución de q1_time:***

Se realiza la ejecición de de `q1_time` con `test_file_path` como argumento (base de datos) y se imprime su resultado.

***3. Impresión de resultado "Humanizado":***

Se imprime el mismo resultado anterior organizando la información para que resulta más facil de comprender.

In [34]:
import time

# Medir el tiempo de ejecución de q1_time
start_time = time.time()
results = q1_time(test_file_path)
end_time = time.time()
# Calcular el tiempo de ejecución
execution_time = end_time - start_time
print(f"Tiempo de ejecución de q1_time: {execution_time} s \n")

# Imprimir los resultados obtenidos
print("Resultados obtenidos:")
print(results, "\n")

print("Resultados humanizados:")
for i, (date, username) in enumerate(results, start=1):
    print(f"{i}. Fecha: {date}, Usuario con más publicaciones: {username}")

Tiempo de ejecución de q1_time: 9.109431982040405 s 

Resultados obtenidos:
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')] 

Resultados humanizados:
1. Fecha: 2021-02-12, Usuario con más publicaciones: RanbirS00614606
2. Fecha: 2021-02-13, Usuario con más publicaciones: MaanDee08215437
3. Fecha: 2021-02-17, Usuario con más publicaciones: RaaJVinderkaur
4. Fecha: 2021-02-16, Usuario con más publicaciones: jot__b
5. Fecha: 2021-02-14, Usuario con más publicaciones: rebelpacifist
6. Fecha: 2021-02-18, Usuario con más publicaciones: neetuanjle_nitu
7. Fecha: 2021-02-15, Usuario con más 

### ***Conclusión*** `q1_time`

Luego de muchas pruebas se opta por seleccionar el metodo que no hace uso de librerias ni funciones adicionales para la implementación de `q1_time` ya que arroja tiempos más bajos en los conteos de fechas y usarios que con otras implementaciones que usan clases como `Counter` y `defaultdict`.

# **R#1 - Enfoque 2:** Optimización de la memoria en uso.

### ***- Primer intento:*** función `q1_memory`

La función `q1_memory` procesa un archivo JSON que contiene registros de tweets y devuelve las 10 fechas más comunes junto con el usuario más activo para cada fecha.

En este primer intento no voy a usar librerias adicionales, utilizo simplemente un diccionario estándar `dates_dict` de Python para ir recolectando fechas y usuarios junto con el método `setdefault()`.

La función `q1_memory_attemp_1` procesa un archivo JSON que contiene registros de tweets y devuelve las 10 fechas más comunes junto con el usuario más activo para cada fecha.

***Descripción detallada:***

***1.   Inicialización del diccionario de fechas y usuarios:***
*   Se crea un diccionario llamado `dates_dict` para almacenar los pares de fecha-usuario y contar la cantidad de tweets por usuario en cada fecha.

***2.   Procesamiento de los datos:***
*   Se abre el archivo especificado en `file_path` en modo de lectura y se procesa línea por línea.
*   Para cada línea, se carga el JSON y se extraen la fecha y el nombre de usuario del tweet.
*   Se actualiza el contador de tweets por usuario y fecha en el diccionario `dates_dict`.

***3.   Ordenamiento de las fechas según la cantidad total de tweets:***
*   Se obtienen las fechas más comunes ordenando el diccionario `dates_dict` según la suma de los valores (número de tweets) de cada fecha.

***4.   Obtención del usuario más activo por fecha:***
*   Para cada fecha más común, se obtiene el usuario más activo (el que más tweets ha realizado) utilizando el método `max`.

***5.   Formateo de las fechas:***
*   Las fechas extraídas inicialmente están en formato de cadena de texto. Se utilizan para crear objetos `datetime.date` utilizando la función `datetime.strptime()` para convertirlas al formato deseado.

***6.   Retorno de resultados:***
*   La función devuelve una lista de tuplas, donde cada tupla contiene una fecha y el usuario más activo para esa fecha.

Esta implementación proporciona una manera eficiente de analizar datos de tweets y obtener información sobre las fechas y usuarios más activos en la plataforma.


In [35]:
import json
from datetime import datetime
from typing import List, Tuple

def q1_memory_attempt_1(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función procesa un archivo JSON que contiene registros de tweets y devuelve las 10 fechas más comunes junto con el usuario más activo para cada fecha.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los registros de tweets.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas, donde cada tupla contiene una fecha y el usuario más activo para esa fecha.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """
    # Paso 1: Inicialización del diccionario de fechas y usuarios
    dates_dict = {}

    # Paso 2: Procesamiento de los datos
    with open(file_path, 'r') as data:
        for line_data in data:
            tweet = json.loads(line_data)
            tweet_date = tweet['date'].split('T')[0]
            username = tweet['user']['username']

            # Paso 3: Actualización del contador de tweets por usuario y fecha
            if tweet_date not in dates_dict:
                dates_dict[tweet_date] = {}

            dates_dict[tweet_date].setdefault(username, 0)
            dates_dict[tweet_date][username] += 1

    # Paso 4: Ordenamiento de las fechas según la cantidad total de tweets
    top_dates = sorted(dates_dict.keys(), key=lambda x: sum(dates_dict[x].values()), reverse=True)[:10]

    # Paso 5: Obtención del usuario más activo por fecha
    top_users = [max(dates_dict[date], key=dates_dict[date].get) for date in top_dates]

    # Paso 6: Formateo de las fechas
    top_dates = [datetime.strptime(date_str, "%Y-%m-%d").date() for date_str in top_dates]

    # Paso 7: Retorno de resultados
    return list(zip(top_dates, top_users))

### ***- Segundo intento:*** función `q1_memory`

En este otro intento, se proporciona una manera eficiente de analizar datos de tweets y obtener información sobre las fechas y usuarios más activos en la plataforma.

La gran diferencia con el anterior es que al realizar la lectura del archivo, los datos se almacenan en una lista donde cada elemento es otra lista que contiene la fecha del tweet y el nombre de usuario del autor.

***Descripción detallada:***

***1.   Lectura del archivo JSON:***
*   La función comienza abriendo el archivo JSON especificado en el parámetro `file_path` en modo de lectura.
*   Lee línea por línea del archivo y carga los datos en un diccionario donde cada clave es la fecha del tweet y el valor es otro diccionario que almacena los usuarios y sus respectivos conteos de tweets.

***2.   Procesamiento de los datos:***
*   Para cada línea en el archivo, se extrae la fecha del tweet y el nombre de usuario del autor.
*   Se actualiza el contador de tweets por usuario y fecha en el diccionario utilizando `setdefault()`.

***3.   Determinación de las fechas más comunes:***
*   Las fechas se ordenan según el número total de tweets publicados en cada una de ellas, utilizando `sorted()` y una función `key`.

***4.   Obtención del usuario más activo por fecha:***
*   Para cada una de las fechas más comunes, se determina el usuario más activo encontrando el usuario con el mayor número de tweets para esa fecha en particular.

***5.   Formateo de las fechas:***
*   Las fechas extraídas inicialmente están en formato de cadena de texto.
*   Se utilizan para crear objetos `datetime.date` utilizando la función `datetime.strptime()` para convertirlas al formato deseado.

***6.   Agrupación de las fechas y usuarios:***
*   Se combinan las fechas formateadas y los usuarios más activos en una lista de tuplas.
*   Cada tupla contiene una fecha y el usuario más activo para esa fecha.

***7.   Retorno de resultados:***
*   La función devuelve la lista de tuplas que contiene las 10 fechas principales con los usuarios más activos para cada una de esas fechas.

In [36]:
import json
from datetime import datetime
from typing import List, Tuple

def q1_memory_attempt_2(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función procesa un archivo JSON que contiene registros de tweets y devuelve las 10 fechas más comunes junto con el usuario más activo para cada fecha.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los registros de tweets.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas, donde cada tupla contiene una fecha y el usuario más activo para esa fecha.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """
    # Paso 1: Inicialización del diccionario de fechas y usuarios
    dates_dict = {}

    # Paso 2: Procesamiento de los datos
    with open(file_path, 'r') as data:
        tweets_dates_users = [[json.loads(line)['date'].split('T')[0], json.loads(line)['user']['username']] for line in data.readlines()]

    for tweet_date, username in tweets_dates_users:
        # Paso 3: Actualización del contador de tweets por usuario y fecha
        if tweet_date not in dates_dict:
            dates_dict[tweet_date] = {}
        dates_dict[tweet_date][username] = dates_dict[tweet_date].get(username, 0) + 1

    # Paso 4: Ordenamiento de las fechas según la cantidad total de tweets
    top_dates = sorted(dates_dict.keys(), key=lambda x: sum(dates_dict[x].values()), reverse=True)[:10]

    # Paso 5: Obtención del usuario más activo por fecha
    top_users = [max(dates_dict[date], key=dates_dict[date].get) for date in top_dates]

    # Paso 6: Formateo de las fechas
    top_dates = [datetime.strptime(date_str, "%Y-%m-%d").date() for date_str in top_dates]

    # Paso 7: Retorno de resultados
    return list(zip(top_dates, top_users))

### ***- Intento definitivo:*** función `q1_memory`

Luego de intentar con los diccionarios estándar de Python, la solución más obvia a este problema es usar la clase `defaultdict` que se encuentra en el módulo `collections` de Python. Funciona de manera similar a un diccionario convencional (dict), pero con una diferencia clave: automáticamente crea valores por defecto para claves que aún no están en el diccionario. Esto significa que no es necesario preocuparse por verificar si una clave existe antes de acceder a ella o asignarle un valor.

En esta implementación también se lee el archivo linea por linea para mejorar el uso de memoria y no cargar todo el archivo json en memoria.


***Descripción detallada:***

***1.   Lectura del archivo JSON:***
*   La función comienza abriendo el archivo JSON especificado en el parámetro `file_path` en modo de lectura.
*   Luego lee todas las líneas del archivo y carga los datos en una lista de listas.
*   Cada sublista contiene la fecha del tweet y el nombre de usuario del autor del tweet.
*   Estos datos se extraen del JSON utilizando la función `json.loads(line)['date'].split('T')[0]` para obtener la fecha y `json.loads(line)['user']['username']` para obtener el nombre de usuario.

***2.   Procesamiento de los datos:***
*   Se utiliza un diccionario para almacenar los pares fecha - usuario y sus respectivos conteos de tweets.
*   Para cada tweet en el archivo, se extrae la fecha y el nombre de usuario.
*   Luego, se actualiza el contador de tweets por usuario y fecha en el diccionario.

***3.   Determinación de las fechas más comunes:***
*   Utilizando `sorted()` y `key`, se determinan las 10 fechas más comunes en el diccionario.
*   Las fechas se ordenan según el número total de tweets publicados en cada una de ellas.

***4.   Obtención del usuario más activo por fecha:***
*   Para cada una de las fechas más comunes, se determina el usuario más activo.
*   Esto se logra encontrando el usuario con el mayor número de tweets para esa fecha en particular.

***5.   Formateo de las fechas:***
*   Las fechas extraídas inicialmente están en formato de cadena de texto.
*   Se utilizan para crear objetos datetime.date utilizando la función `datetime.strptime()` para convertirlas al formato deseado.

***6.   Agrupación de las fechas y usuarios:***
*   Finalmente, se combinan las fechas formateadas y los usuarios más activos en una lista de tuplas.
*   Cada tupla contiene una fecha y el usuario más activo para esa fecha.

***7.   Retorno de resultados:***
*   La función devuelve la lista de tuplas que contiene las 10 fechas principales con los usuarios más activos para cada una de esas fechas.
*   Esta función proporciona una manera eficiente de analizar datos de tweets y obtener información sobre las fechas y usuarios más activos en la plataforma.

In [37]:
import json
from collections import defaultdict
from datetime import datetime
from typing import List, Tuple

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función analiza un archivo JSON que contiene registros de tweets y devuelve las 10 fechas
    más comunes junto con el usuario más activo para cada fecha.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los registros de tweets.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas, donde cada tupla contiene una fecha
        y el usuario más activo para esa fecha.

    Raises:
        FileNotFoundError: Si no se encuentra el archivo especificado en file_path.
    """

    # Paso 1: Lectura del archivo JSON
    dates_dict = defaultdict(lambda: defaultdict(int))

    # Paso 2: Procesamiento de los datos
    with open(file_path, 'r') as data:
        for line_data in data:
            tweet = json.loads(line_data)
            tweet_date = tweet['date'].split('T')[0]
            username = tweet['user']['username']

            # Actualización del contador de tweets por usuario y fecha
            dates_dict[tweet_date][username] += 1

    # Paso 3: Determinación de las fechas más comunes
    top_dates = sorted(dates_dict.keys(), key=lambda x: sum(dates_dict[x].values()), reverse=True)[:10]

    # Paso 4: Obtención del usuario más activo por fecha
    top_users = [max(dates_dict[date], key=dates_dict[date].get) for date in top_dates]

    # Paso 5: Formateo de las fechas
    top_dates = [datetime.strptime(date_str, "%Y-%m-%d").date() for date_str in top_dates]

    # Paso 6: Agrupación de las fechas y usuarios
    result = list(zip(top_dates, top_users))

    return result

## - Ejecuto la función `q1_memory`

***Medición de memoria:***

Utilizando memory_profiler para medir el uso de memoria de la función `q1_memory`. `memory_usage` devuelve una lista que contiene el uso de memoria en diferentes puntos de la ejecución de la función. En este caso, se pasa `(q1_memory, (test_file_path,))` como argumento a `memory_usage`, lo que significa que estás midiendo el uso de memoria de `q1_memory` con `test_file_path` como argumento (base de datos). Finalmente se imprime el usuo de memoria de `q1_memory`.

***2. Ejecución de q1_memory:***

Se realiza la ejecición de de `q1_memory` con `test_file_path` como argumento (base de datos) y se imprime su resultado.

***3. Impresión de resultado "Humanizado":***

Se imprime el mismo resultado anterior organizando la información para que resulta más facil de comprender.


In [38]:
import memory_profiler

# Medir el uso de memoria de q1_memory con memory-profiler
mem_usage = memory_profiler.memory_usage((q1_memory, (test_file_path, )))

# Imprimir el uso de memoria
print(f"Uso de memoria de q1_memory: {max(mem_usage)} MB \n")

# Ejecutar q1_memory y obtener los resultados
results = q1_memory(test_file_path)

# Imprimir los resultados obtenidos
print("Resultados obtenidos:")
print(results, "\n")

print("Resultados humanizados:")
for i, (date, username) in enumerate(results, start=1):
    print(f"{i}. Fecha: {date}, Usuario con más publicaciones: {username}")

Uso de memoria de q1_memory: 313.60546875 MB 

Resultados obtenidos:
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')] 

Resultados humanizados:
1. Fecha: 2021-02-12, Usuario con más publicaciones: RanbirS00614606
2. Fecha: 2021-02-13, Usuario con más publicaciones: MaanDee08215437
3. Fecha: 2021-02-17, Usuario con más publicaciones: RaaJVinderkaur
4. Fecha: 2021-02-16, Usuario con más publicaciones: jot__b
5. Fecha: 2021-02-14, Usuario con más publicaciones: rebelpacifist
6. Fecha: 2021-02-18, Usuario con más publicaciones: neetuanjle_nitu
7. Fecha: 2021-02-15, Usuario con más publica

### ***Conclusión*** `q1_memory`
Luego de todas las pruebas, para este punto en el que se prioriza el uso de memoria, se opta por dejar la implementación de la función `q1_memory` en la que se realiza la implementación de `defaultdict` y la lectura del JSON linea por linea, ya que es la que mejores resultados arroja.



---



# **Reto #2**

Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:
```python
def q2_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns:
[("✈️", 6856), ("❤️", 5876), ...]
```

# **R#2 - Enfoque 1:** Optimización del tiempo de ejecución.

### ***- Primer intento:*** función `q2_time_attempt_1`

En este intento lo que hago es buscar los emojis mapeando los caracteres y usando un diccionario estándar para contar la frecuencia de cada emoji. Aca se utiliza el método `json.load(f)` para cargar todos los tweets del archivo JSON en una lista de diccionarios.



#### Descripción detallada:

1. **Inicialización del diccionario de conteo de emojis:**
   - Se crea un diccionario vacío llamado `emoji_counts` para almacenar la frecuencia de cada emoji.

2. **Lectura del archivo JSON y procesamiento de los tweets:**
   - Se abre el archivo JSON especificado en modo de lectura.
   - Se leen todos los tweets de una vez y se almacenan en la lista `tweets`.
   
3. **Obtención del contenido de los tweets en un gran string:**
   - Se inicializa una cadena vacía llamada `tweets_content` para almacenar el contenido de todos los tweets.
   - Se itera sobre cada tweet en la lista `tweets`, se carga el contenido del tweet como un diccionario JSON y se agrega el contenido al string `tweets_content`.

4. **Búsqueda de emojis en el gran string:**
   - Se recorre cada caracter del string `tweets_content` y se verifica si es un emoji válido utilizando la función `map(chr, range(128, 1024))`.
   - Los emojis válidos encontrados se agregan a la lista `all_emojis`.
   
5. **Conteo de la frecuencia de cada emoji:**
   - Se itera sobre cada emoji en la lista `all_emojis`.
   - Se actualiza el diccionario `emoji_counts` con la frecuencia de cada emoji.
   
6. **Obtención de los 10 emojis más utilizados:**
   - Se ordena el diccionario `emoji_counts` según el valor de frecuencia de los emojis en orden descendente.
   - Se seleccionan los 10 emojis más utilizados y se devuelven en una lista de tuplas.

7. **Retorno de resultados:**
   - La función devuelve una lista de tuplas que contiene los 10 emojis más utilizados junto con su respectivo conteo de ocurrencias.


In [39]:
import json
from typing import List, Tuple

def q2_time_attempt_1(file_path: str) -> List[Tuple[str, int]]:
    """
    Encuentra los 10 emojis más utilizados en los tweets presentes en el archivo JSON especificado,
    junto con su respectivo conteo de ocurrencias.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 emojis más utilizados, cada una con su respectivo conteo de ocurrencias.
    """
    # Paso 1: Inicialización de un diccionario para contar la frecuencia de cada emoji
    emoji_counts = {}

    # Paso 2: Lectura del archivo JSON y procesamiento de los tweets
    with open(file_path, 'r') as data:
        # Paso 3: Se obtienen todos los objetos de tweets de una vez
        tweets = data.readlines()

    # Paso 4: Se obtiene un gran string que contiene todos los contenidos de los tweets
    tweets_content = ""
    for tweet in tweets:
        tweets_content += json.loads(tweet)['content']

    # Paso 5: Obtención de todos los emojis presentes en el gran string
    all_emojis = [char for char in tweets_content if char in ' '.join(map(chr, range(128, 1024)))]

    # Paso 6: Conteo de la frecuencia de cada emoji y actualización del diccionario de conteo
    for emoji in all_emojis:
        emoji_counts[emoji] = emoji_counts.get(emoji, 0) + 1

    # Paso 7: Obtención de los 10 emojis más utilizados con su respectivo conteo
    top_10_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    return top_10_emojis

### ***- Segundo intento:*** función `q2_time`

A diferencia de la implementación anterior, en donde busco los emojis comparando si están dentro del rango de códigos Unicode que representan emojis lo cual se prueba que es muy ineficiente, ahora en este intento utilizo la función `emoji_list` de la librería `emoji` para obtener los emojis presentes en el contenido de cada tweet, manteniendo el resto de la funcionalidad intacta.


#### Descripción detallada:

***1.   Inicialización del contador de emojis:***
*   Se inicializa un diccionario vacío `emoji_counts` para contar la frecuencia de cada emoji encontrado en los tweets.

***2.   Lectura del archivo JSON:***
*   Se lee el archivo JSON especificado en el parámetro `file_path` y se almacenan todos los objetos de tweets de una vez en la lista `tweets`.

***3.   Concatenación de los contenidos de los tweets:***
*   Se recorren todos los tweets en la lista `tweets` y se concatenan sus contenidos en un solo gran string `tweets_content`.

***4.   Obtención de los emojis presentes en los tweets:***
*   Utilizando la función `emoji_list` se obtienen los emojis presentes en el gran string `tweets_content`. La función `emoji_list` devuelve una lista de diccionarios, donde cada diccionario contiene información sobre un emoji, incluyendo su representación Unicode.
*   `[emoji['emoji'] for emoji in emoji_list(tweets_content)]` Utiliza una list comprehension para extraer solo la representación Unicode de cada emoji de la lista de diccionarios devuelta por `emoji_list`, y almacena estas representaciones en la lista `emojis_in_tweets`. Esto asegura que solo se obtenga la representación Unicode de cada emoji y no se incluyan otros detalles del diccionario.

***5.   Actualización del contador de emojis:***
*   Se recorren todos los emojis presentes en la lista `emojis_in_tweets` y se actualiza el contador de emojis `emoji_counts` con la frecuencia de cada emoji.

***6.   Obtención de los 10 emojis más utilizados:***
*   Se ordena el diccionario `emoji_counts` por sus valores en orden descendente para obtener los emojis más utilizados.
*   Se seleccionan los primeros 10 elementos del diccionario ordenado para obtener los 10 emojis más utilizados junto con su respectivo conteo de ocurrencias.

***7.   Retorno de resultados:***
*   La función devuelve una lista de tuplas que contiene los 10 emojis más utilizados, donde cada tupla tiene la forma `(emoji, conteo)`.


In [40]:
import json
from emoji import emoji_list
from typing import List, Tuple

def q2_time_attempt_2(file_path: str) -> List[Tuple[str, int]]:
    """
    Encuentra los 10 emojis más utilizados en los tweets presentes en el archivo JSON especificado,
    junto con su respectivo conteo de ocurrencias.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 emojis más utilizados, cada una con su respectivo conteo de ocurrencias.
    """
    # Paso 1: Se inicializa un diccionario para contar la frecuencia de cada emoji
    emoji_counts = {}

    # Paso 2: Se obtienen todos los objetos de tweets de una vez
    with open(file_path, 'r') as data:
        tweets = data.readlines()

    # Paso 3: Se obtiene un gran string que contiene todos los contenidos de los tweets
    tweets_content = ""
    for tweet in tweets:
        tweets_content += json.loads(tweet)['content']

    # Paso 4: Se obtienen los emojis presentes en el gran string utilizando emoji_list
    emojis_in_tweets = [emoji['emoji'] for emoji in emoji_list(tweets_content)]

    # Paso 5: Se actualiza el contador de emojis con la frecuencia de cada emoji
    for emoji in emojis_in_tweets:
        emoji_counts[emoji] = emoji_counts.get(emoji, 0) + 1

    # Paso 6: Se obtienen los 10 emojis más utilizados con su respectivo conteo
    top_10_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    return top_10_emojis

### ***- Intento definitivo:*** función `q2_time`


En esta versión final, se utiliza la clase `Counter`, que es una clase en Python que se encuentra en el módulo `collections` y que me permite contar la ocurrencia de elementos en una secuencia y generar un diccionario con las frecuencias de esos elementos para contar la ocurrencia de emojis.

En lugar de iterar manualmente sobre los tweets y acumular los emojis, se utiliza un enfoque más eficiente. El uso de Counter simplifica significativamente el proceso y mejora la eficiencia del código en términos de legibilidad y rendimiento.


#### Detalles de la Implementación

1. **Inicialización del contador de emojis:**
   Se inicializa un objeto `Counter` para contar la frecuencia de cada emoji en los tweets.

2. **Lectura del archivo JSON y procesamiento de los tweets:**
   Se abre el archivo JSON especificado en modo de lectura y se recorren todas las líneas.
   Para cada línea, se carga el contenido del tweet como un diccionario JSON y se extraen los emojis presentes en el contenido del tweet.

3. **Obtención de los emojis de cada línea:**
   Se utiliza la función `emoji_list` para obtener una lista de emojis presentes en el contenido del tweet.

4. **Actualización del contador de emojis:**
   Se actualiza el contador de emojis utilizando el método `update` del objeto `Counter` con la lista de emojis obtenida en el paso anterior.

5. **Devolución de los 10 emojis más utilizados:**
   Se devuelve una lista con los 10 emojis más utilizados junto con sus respectivos conteos de ocurrencias, utilizando el método `most_common` del objeto `Counter`.

In [41]:
import json
from collections import Counter
from datetime import datetime
from emoji import emoji_list
from typing import List, Tuple

def q2_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función procesa un archivo JSON que contiene registros de tweets y devuelve los 10 emojis más utilizados,
    junto con su respectivo conteo de ocurrencias.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 emojis más utilizados,
        cada una con su respectivo conteo de ocurrencias.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """

    # Paso 1: Inicialización del contador de emojis
    emoji_counter = Counter()

    # Paso 2: Lectura del archivo JSON y procesamiento de los tweets
    with open(file_path, 'r') as data:
        for line in data:
            tweet_content = json.loads(line)['content']

            # Paso 3: Obtención de los emojis de cada línea
            tweet_emojis = [emoji['emoji'] for emoji in emoji_list(tweet_content)]

            # Paso 4: Actualización del contador de emojis
            emoji_counter.update(tweet_emojis)

    # Paso 5: Devolución de los 10 emojis más utilizados
    return emoji_counter.most_common(10)

## - Ejecución de la función `q2_time`

Al igual que en el primer reto, solo se utiliza la libreria `time` para la medición del tiempo que tarda en ejecutarse la funcion `q2_time`.

***1. Medición de tiempo:***

Dada la explicación anterior, para la medición del tiempo de ejecución de `q2_time` solamente utilizo la función `time` nativa de Phyton para tomar un tiempo de inicio antes de la ejecución de la función y un tiempo final una vez culmina su ejecución luego calculo la diferencia entre ambos y obtengo la medición de tiempo requerida.

***2. Ejecución de q2_time:***

Se realiza la ejecición de de `q2_time` con `test_file_path` como argumento (base de datos) y se imprime su resultado.

***3. Impresión de resultado "Humanizado":***

Se imprime el mismo resultado anterior organizando la información para que resulta más facil de comprender.

In [42]:
import time

# Medir el tiempo de ejecución de q2_time
start_time = time.time()
results = q2_time(test_file_path)
end_time = time.time()
# Calcular el tiempo de ejecución
execution_time = end_time - start_time
print(f"Tiempo de ejecución de q2_time: {execution_time} s \n")

# Imprimir los resultados obtenidos
print("Resultados obtenidos:")
print(results, "\n")

print("Resultados humanizados:")
for i, (date, username) in enumerate(results, start=1):
    print(f"{i}. Fecha: {date}, Usuario con más publicaciones: {username}")

Tiempo de ejecución de q2_time: 44.589800119400024 s 

Resultados obtenidos:
[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)] 

Resultados humanizados:
1. Fecha: 🙏, Usuario con más publicaciones: 5049
2. Fecha: 😂, Usuario con más publicaciones: 3072
3. Fecha: 🚜, Usuario con más publicaciones: 2972
4. Fecha: 🌾, Usuario con más publicaciones: 2182
5. Fecha: 🇮🇳, Usuario con más publicaciones: 2086
6. Fecha: 🤣, Usuario con más publicaciones: 1668
7. Fecha: ✊, Usuario con más publicaciones: 1651
8. Fecha: ❤️, Usuario con más publicaciones: 1382
9. Fecha: 🙏🏻, Usuario con más publicaciones: 1317
10. Fecha: 💚, Usuario con más publicaciones: 1040


### ***Conclusión*** `q2_time`

Los intentos 1 y 2 se intentaron hacer utilizando el minimo de funciones adicionales, en el primero de ellos usando representación Unicode de cada emoji para hacer la búsqueda y en el segundo usando `emoji_list()`, pero al revisar su ejecución y los resultados que se obtienen no son satisfactorios, puesto que muchos Unicodes los malinterpreta y el proceso de búsqueda es muy lento.

En la versión final se utilizan `Counter` y `emoji_list` de manera que se crea una manera simple y eficiente de hacer la búsqueda y conteo de los emojis sobre la base de datos y después de todos los intentos se obtiene en esta implementación el mejor tiempo de respuesta.

# **R#2 - Enfoque 2:** Optimización de la memoria en uso.


### ***- Primer intento:*** función `q2_memory`

En este primer intento se toma el primer intento realizado en `q2_time`, pero para optimizar el uso de memoria en esta función se procesa cada tweet línea por línea directamente desde el archivo, evitando cargar todo el contenido del archivo en memoria.

En este intento, procesamos cada línea del archivo directamente en el bucle `for line in data`. Esto reduce la carga de memoria al no necesitar almacenar todas las líneas en una lista antes de procesarlas.


#### Descripción detallada:

1. **Inicialización del diccionario de conteo de emojis:**
   - Se crea un diccionario vacío llamado `emoji_counts` para almacenar la frecuencia de cada emoji.

2. **Lectura del archivo JSON y procesamiento de los tweets:**
   - Se abre el archivo JSON especificado en modo de lectura.
   - Se leen todos los tweets de una vez y se almacenan en la lista `tweets`.
   
3. **Obtención del contenido de los tweets en un gran string:**
   - Se inicializa una cadena vacía llamada `tweets_content` para almacenar el contenido de todos los tweets.
   - Se itera sobre cada tweet en la lista `tweets`, se carga el contenido del tweet como un diccionario JSON y se agrega el contenido al string `tweets_content`.

4. **Búsqueda de emojis en el gran string:**
   - Se recorre cada caracter del string `tweets_content` y se verifica si es un emoji válido utilizando la función `map(chr, range(128, 1024))`.
   - Los emojis válidos encontrados se agregan a la lista `all_emojis`.
   
5. **Conteo de la frecuencia de cada emoji:**
   - Se itera sobre cada emoji en la lista `all_emojis`.
   - Se actualiza el diccionario `emoji_counts` con la frecuencia de cada emoji.
   
6. **Obtención de los 10 emojis más utilizados:**
   - Se ordena el diccionario `emoji_counts` según el valor de frecuencia de los emojis en orden descendente.
   - Se seleccionan los 10 emojis más utilizados y se devuelven en una lista de tuplas.

7. **Retorno de resultados:**
   - La función devuelve una lista de tuplas que contiene los 10 emojis más utilizados junto con su respectivo conteo de ocurrencias.



In [43]:
import json
from typing import List, Tuple

def q2_memory_attempt_1(file_path: str) -> List[Tuple[str, int]]:
    """
    Encuentra los 10 emojis más utilizados en los tweets presentes en el archivo JSON especificado,
    junto con su respectivo conteo de ocurrencias.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 emojis más utilizados, cada una con su respectivo conteo de ocurrencias.
    """
    # Paso 1: Inicialización de un diccionario para contar la frecuencia de cada emoji
    emoji_counts = {}

    # Paso 2: Lectura del archivo JSON y procesamiento de los tweets línea por línea
    with open(file_path, 'r') as data:
        for line in data:
            # Paso 3: Cargar el contenido del tweet como un diccionario
            tweet = json.loads(line)

            # Paso 4: Obtener los emojis presentes en el contenido del tweet
            tweet_content = tweet['content']
            emojis_in_tweet = [char for char in tweet_content if char in ' '.join(map(chr, range(128, 1024)))]

            # Paso 5: Actualización del contador de emojis con la frecuencia de cada emoji en el tweet
            for emoji in emojis_in_tweet:
                emoji_counts[emoji] = emoji_counts.get(emoji, 0) + 1

    # Paso 6: Obtención de los 10 emojis más utilizados con su respectivo conteo
    top_10_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    return top_10_emojis


### ***- Segundo intento:*** función `q2_memory`

En este intento de `q2_memory` tomo como base el correspondiente intento de `q2_time` en donde utilizo `emoji_list()` para hacer la búsqueda de los emojis y se hace la lectura del archivo JSON y procesamiento de los tweets línea por línea para optimizar el manejo de memoria.


#### Descripción detallada:

1. **Inicialización del diccionario de conteo de emojis:**
   - Se crea un diccionario vacío llamado `emoji_counts` para almacenar la frecuencia de cada emoji.

2. **Lectura del archivo JSON y procesamiento de los tweets línea por línea:**
   - Se abre el archivo JSON especificado en modo de lectura.
   - Se itera sobre cada línea del archivo para procesar cada tweet individualmente.

3. **Cargar el contenido del tweet como un diccionario:**
   - Se carga el contenido del tweet como un diccionario JSON utilizando la función `json.loads()`.

4. **Obtención de los emojis presentes en el contenido del tweet:**
   - Se extrae el contenido del tweet y se obtienen los emojis presentes en él utilizando la función `emoji_list()` del módulo `emoji`.
   - Los emojis se almacenan en una lista llamada `emojis_in_tweet`.

5. **Actualización del contador de emojis con la frecuencia de cada emoji en el tweet:**
   - Se itera sobre cada emoji en la lista `emojis_in_tweet`.
   - Se actualiza el diccionario `emoji_counts` con la frecuencia de cada emoji.

6. **Obtención de los 10 emojis más utilizados:**
   - Se ordena el diccionario `emoji_counts` según el valor de frecuencia de los emojis en orden descendente.
   - Se seleccionan los 10 emojis más utilizados y se devuelven en una lista de tuplas.

Este intento utiliza la función `emoji_list()` del módulo `emoji` para obtener los emojis presentes en el contenido de cada tweet, sin la necesidad de usar ninguna librería adicional.


In [44]:
import json
from typing import List, Tuple
from emoji import emoji_list

def q2_memory_attempt_2(file_path: str) -> List[Tuple[str, int]]:
    """
    Encuentra los 10 emojis más utilizados en los tweets presentes en el archivo JSON especificado,
    junto con su respectivo conteo de ocurrencias.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 emojis más utilizados, cada una con su respectivo conteo de ocurrencias.
    """
    # Paso 1: Inicialización de un diccionario para contar la frecuencia de cada emoji
    emoji_counts = {}

    # Paso 2: Lectura del archivo JSON y procesamiento de los tweets línea por línea
    with open(file_path, 'r') as data:
        for line in data:
            # Paso 3: Cargar el contenido del tweet como un diccionario
            tweet = json.loads(line)

            # Paso 4: Obtener los emojis presentes en el contenido del tweet
            tweet_content = tweet['content']
            emojis_in_tweet = [emoji['emoji'] for emoji in emoji_list(tweet_content)]

            # Paso 5: Actualización del contador de emojis con la frecuencia de cada emoji en el tweet
            for emoji in emojis_in_tweet:
                emoji_counts[emoji] = emoji_counts.get(emoji, 0) + 1

    # Paso 6: Obtención de los 10 emojis más utilizados con su respectivo conteo
    top_10_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    return top_10_emojis


### ***- Intento definitivo:*** función `q2_memory`

Este enfoque garantiza un procesamiento eficiente del archivo JSON al hacerlo linea por linea garantizando menor consumo de memoria y permite encontrar rápidamente los emojis más utilizados en los tweets.


#### Detalles de la implementación:

1. **Inicialización del contador de emojis:**
   - Se utiliza la clase `Counter` del módulo `collections` para inicializar un contador que almacenará el conteo de ocurrencias de cada emoji.

2. **Lectura del archivo JSON y procesamiento de los tweets:**
   - Se utiliza la función `open()` para abrir el archivo JSON en modo de lectura.
   - Se utiliza un loop `for` para iterar sobre cada línea del archivo JSON.
   - Se utiliza la función `json.loads()` para cargar cada línea como un objeto JSON y acceder al contenido del tweet.

3. **Obtención de los emojis de cada línea:**
   - Se utiliza la función `emoji_list()` del módulo `emoji` para obtener una lista de emojis presentes en el contenido del tweet.

4. **Actualización del contador de emojis:**
   - Se utiliza una lista de comprensión para obtener los emojis de cada línea del tweet.
   - Se utiliza el método `update()` del objeto `Counter` para actualizar el contador con los emojis obtenidos.

5. **Devolución de los 10 emojis más utilizados:**
   - Se utiliza el método `most_common(10)` del objeto `Counter` para obtener los 10 emojis más utilizados junto con su respectivo conteo de ocurrencias.


In [45]:
import json
from collections import Counter
from datetime import datetime
from emoji import emoji_list
from typing import List, Tuple

def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función procesa un archivo JSON que contiene registros de tweets y devuelve los 10 emojis más utilizados,
    junto con su respectivo conteo de ocurrencias.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 emojis más utilizados,
        cada una con su respectivo conteo de ocurrencias.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """

    # Paso 1: Inicialización del contador de emojis
    emoji_counter = Counter()

    # Paso 2: Lectura del archivo JSON y procesamiento de los tweets
    with open(file_path, 'r') as data:
        for line in data:
            tweet_content = json.loads(line)['content']

            # Paso 3: Obtención de los emojis de cada línea
            tweet_emojis = [emoji['emoji'] for emoji in emoji_list(tweet_content)]

            # Paso 4: Actualización del contador de emojis
            emoji_counter.update(tweet_emojis)

    # Paso 5: Devolución de los 10 emojis más utilizados
    return emoji_counter.most_common(10)


## - Ejecución de la función `q2_memory`

***Medición de memoria:***

Utilizando memory_profiler para medir el uso de memoria de la función `q2_memory`. `memory_usage` devuelve una lista que contiene el uso de memoria en diferentes puntos de la ejecución de la función. En este caso, se pasa `(q2_memory, (test_file_path,))` como argumento a `memory_usage`, lo que significa que estás midiendo el uso de memoria de `q2_memory` con `test_file_path` como argumento (base de datos). Finalmente se imprime el usuo de memoria de `q2_memory`.

***2. Ejecución de q2_memory:***

Se realiza la ejecición de de `q2_memory` con `test_file_path` como argumento (base de datos) y se imprime su resultado.

***3. Impresión de resultado "Humanizado":***

Se imprime el mismo resultado anterior organizando la información para que resulta más facil de comprender.

In [46]:
import memory_profiler

# Medir el uso de memoria de q1_memory con memory-profiler
mem_usage = memory_profiler.memory_usage((q2_memory, (test_file_path, )))

# Imprimir el uso de memoria
print(f"Uso de memoria de q1_memory: {max(mem_usage)} MB \n")

# Ejecutar q1_memory y obtener los resultados
results = q2_memory(test_file_path)

# Imprimir los resultados obtenidos
print("Resultados obtenidos:")
print(results, "\n")

print("Resultados humanizados:")
for i, (date, username) in enumerate(results, start=1):
    print(f"{i}. Fecha: {date}, Usuario con más publicaciones: {username}")

Uso de memoria de q1_memory: 279.41015625 MB 

Resultados obtenidos:
[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)] 

Resultados humanizados:
1. Fecha: 🙏, Usuario con más publicaciones: 5049
2. Fecha: 😂, Usuario con más publicaciones: 3072
3. Fecha: 🚜, Usuario con más publicaciones: 2972
4. Fecha: 🌾, Usuario con más publicaciones: 2182
5. Fecha: 🇮🇳, Usuario con más publicaciones: 2086
6. Fecha: 🤣, Usuario con más publicaciones: 1668
7. Fecha: ✊, Usuario con más publicaciones: 1651
8. Fecha: ❤️, Usuario con más publicaciones: 1382
9. Fecha: 🙏🏻, Usuario con más publicaciones: 1317
10. Fecha: 💚, Usuario con más publicaciones: 1040


### ***Conclusión*** `q2_memory`

Semejante que con q2_time, los intentos 1 y 2 se intentaron hacer utilizando el minimo de funciones adicionales, en el primero de ellos usando representación Unicode de cada emoji para hacer la búsqueda y en el segundo usando `emoji_list()`, pero al revisar su ejecución y los resultados que se obtienen no son satisfactorios, puesto que muchos Unicodes los malinterpreta y el proceso de búsqueda es muy lento.

En la versión final se utilizan `Counter`, `emoji_list()`, junto con un recorrido linea a linea del archivo para evitar la sobrecarga en memoria de información y asi, de esta manera se crea una manera simple y eficiente de hacer la búsqueda y conteo de los emojis sobre la base de datos y después de todos los intentos se obtiene en esta implementación el mejor tiempo de respuesta.



---



# **Reto R#3**


El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. Debe incluir las siguientes funciones:
```python
def q3_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns:
[("LATAM321", 387), ("LATAM_CHI", 129), ...]
```

# **R#3 - Enfoque 1:** Optimización del tiempo de ejecución.

### ***- Primer intento:*** función `q3_time`

En este primer intento de implementar la función `q3_time`, proceso el archivo JSON  y devuelve los 10 nombres de usuario más mencionados junto con su frecuencia de menciones.


#### Descripción detallada:

1. **Lectura del archivo JSON:**
   - Se abre el archivo especificado en `file_path` en modo de lectura (`'r'`) utilizando el contexto `with`.
   - Se intenta parsear las líneas JSON utilizando la biblioteca `json`, extrayendo las listas de usuarios mencionados en cada tweet.
   
2. **Procesamiento de los datos extraídos:**
   - Se inicializa una lista vacía llamada `usernames` para almacenar los nombres de usuario mencionados en los tweets.
   - Se itera sobre cada lista de usuarios mencionados en los tweets extraídos.
   - Si la lista de usuarios no es `None`, se extraen los nombres de usuario de cada usuario y se agregan a la lista `usernames`.

3. **Conteo de menciones de usuario:**
   - Utilizando la clase `Counter` de la biblioteca estándar `collections`, se cuenta la frecuencia de cada nombre de usuario en la lista `usernames`.
   - Se obtienen los 10 nombres de usuario más comunes utilizando el método `most_common(10)` de `Counter`.

4. **Retorno de resultados:**
   - La función devuelve una lista de tuplas que contiene los 10 nombres de usuario más mencionados junto con su respectiva frecuencia de menciones.

In [47]:
import json
from collections import Counter
from typing import List, Tuple

def q3_time_attempt_1(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función procesa un archivo JSON que potencialmente puede contener tweets mal formados y devuelve los 10 nombres de usuario más mencionados junto con su frecuencia de menciones.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 nombres de usuario más mencionados junto con su respectiva frecuencia de menciones.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """

    # Paso 1: Lectura del archivo JSON y extracción de las listas de usuarios mencionados en los tweets
    with open(file_path, 'r') as data:
        try:
            # Se intenta hacer el parsing de las líneas JSON utilizando la biblioteca 'json'
            tweet_mentioned_users = [json.loads(line)['mentionedUsers'] for line in data.readlines()]
        except (json.JSONDecodeError, KeyError) as e:
            # Manejar posibles errores de análisis de JSON o la clave faltante de 'mentionedUsers'
            print(f"Error parsing linea: {e}")
            tweet_mentioned_users = []

    # Paso 2: Procesamiento de los datos extraídos para obtener los nombres de usuario mencionados
    usernames = []
    for user_list in tweet_mentioned_users:
        if user_list is not None:
            # Se extraen los nombres de usuario de cada lista de usuarios mencionados
            usernames.extend([user["username"] for user in user_list])

    # Paso 3: Conteo de menciones de usuario y obtención de los 10 más comunes
    user_mentions = Counter(usernames).most_common(10)

    return list(user_mentions)

### ***- Segundo intento:*** función `q3_time`

En este intento utilizo una lectura linea a linea del JSON y verifico si tiene información de user antes de realizar el conteo con la ayuda de `Counter`.

#### Detalles de la implementación:

1. **Lectura del archivo JSON y extracción de los usuarios mencionados en los tweets:**
   - Se abre el archivo JSON especificado en modo de lectura.
   - Para cada línea del archivo JSON, se intenta cargar el tweet como un objeto JSON.
   - Si el tweet contiene información de usuario (`user`) y dicho usuario no es nulo, se extrae el nombre de usuario (`username`) y se agrega a la lista `mentioned_users`.

2. **Conteo de menciones de usuario y obtención de los 10 más comunes:**
   - Se utiliza la clase `Counter` para contar la frecuencia de cada nombre de usuario en la lista `mentioned_users`.
   - Se obtienen los 10 nombres de usuario más mencionados junto con su respectiva frecuencia de menciones utilizando el método `most_common(10)` de la clase `Counter`.

En caso de que ocurra un error durante la decodificación del JSON (por ejemplo, si hay líneas mal formateadas), se maneja adecuadamente y se continúa con el procesamiento de las líneas restantes del archivo.


In [48]:
import json
from collections import Counter
from typing import List, Tuple

def q3_time_attempt_2(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función procesa un archivo JSON que contiene tweets y devuelve los 10 nombres de usuario
    más mencionados junto con su frecuencia de menciones.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 nombres de usuario más mencionados junto con su respectiva frecuencia de menciones.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """
    # Paso 1: Lectura del archivo JSON y extracción de los usuarios mencionados en los tweets
    with open(file_path, 'r') as data:
        mentioned_users = []

        # Procesar cada línea del archivo JSON
        for line in data:
            tweet = json.loads(line)
            if 'user' in tweet and tweet['user'] is not None and 'username' in tweet['user']:
                mentioned_users.append(tweet['user']['username'])

    # Paso 2: Conteo de menciones de usuario y obtención de los 10 más comunes
    user_mentions = Counter(mentioned_users).most_common(10)

    return user_mentions

### ***- Intento definitivo:*** función `q3_time`


En esta versión final, se utilizan `list comprehensions` para iterar de manera eficiente tanto en la lectura del JSON como en la obtención del usuario teniendo especial cuidado con las listas vacias.

 Finalmente se hace uso de la clase `Counter` de la libreria `collections` para obtener los 10 nombres de usuario más mencionados junto con su respectiva frecuencia de menciones.

 El resultado es una función muy corta pero eficiente para iterar y encontrar los usuarios más mencionados en base de datos.


#### Detalles de la implementación:

1. **Obtención de las listas de usuarios mencionados en cada tweet:**
   - Se utiliza la función `open()` para abrir el archivo JSON especificado en modo de lectura. Esto devuelve un objeto de archivo.
   - Se utiliza la función `readlines()` para leer cada línea del archivo y almacenarlas en una lista.
   - Se utiliza una `list comprehension` para iterar sobre cada línea del archivo JSON.
   - Se utiliza la función `json.loads()` para analizar cada línea como un objeto JSON.
   - Se accede al atributo `'mentionedUsers'` de cada tweet para obtener la lista de usuarios mencionados en ese tweet.

2. **Obtención del nombre de usuario de cada usuario en las listas de menciones:**
   - Nuevamente se usa una `list comprehension` para iterar sobre cada objeto en la lista de menciones de usuarios obtenida en el paso anterior.
   - Se utiliza la función `append()` para agregar el nombre de usuario (`'username'`) de cada objeto a la lista `usernames`.
   - Se utiliza una condición para filtrar las listas vacías de menciones (caso None) y evitar errores al intentar acceder al nombre de usuario.

3. **Obtención de los 10 nombres de usuario más comunes:**
   - Se utiliza la clase `Counter` del módulo `collections` para contar la frecuencia de cada nombre de usuario en la lista `usernames`.
   - Se utiliza el método `most_common(10)` de la clase `Counter` para obtener los 10 nombres de usuario más mencionados junto con su respectiva frecuencia de menciones.
   - Este método devuelve una lista de tuplas, donde cada tupla contiene el nombre de usuario y el recuento de menciones, ordenados por frecuencia de menciones de mayor a menor.


In [49]:
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función procesa un archivo JSON que contiene registros de tweets y devuelve los 10 nombres de usuario
    más mencionados en los tweets, junto con la frecuencia de menciones de cada uno.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 nombres de usuario más mencionados,
        junto con la frecuencia de menciones de cada uno.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """
    # Paso 1: Se obtienen las listas de usuarios mencionados en cada tweet
    with open(file_path, 'r') as data:
        tweet_mentioned_users = [json.loads(line)['mentionedUsers'] for line in data.readlines()]

    # Paso 2: Se obtiene el username de cada user dentro de cada lista de mentionedUsers, exceptuando las listas vacías (caso None).
    usernames = [user['username'] for obj in tweet_mentioned_users if obj is not None for user in obj]

    # Paso 3: Se retornan los 10 usernames más comunes usando la clase Counter para contar y ordenar.
    return Counter(usernames).most_common(10)


## - Pruebas y Ejecución de la función `q3_time`

Al igual que en el primer reto, solo se utiliza la libreria `time` para la medición del tiempo que tarda en ejecutarse la funcion `q3_time`.

***1. Medición de tiempo:***

Dada la explicación anterior, para la medición del tiempo de ejecución de `q3_time` solamente utilizo la función `time` nativa de Phyton para tomar un tiempo de inicio antes de la ejecución de la función y un tiempo final una vez culmina su ejecución luego calculo la diferencia entre ambos y obtengo la medición de tiempo requerida.

***2. Ejecución de q2_time:***

Se realiza la ejecición de de `q3_time` con `test_file_path` como argumento (base de datos) y se imprime su resultado.

***3. Impresión de resultado "Humanizado":***

Se imprime el mismo resultado anterior organizando la información para que resulta más facil de comprender.

In [50]:
import time

# Medir el tiempo de ejecución de q3_time
start_time = time.time()
results = q3_time(test_file_path)
end_time = time.time()
# Calcular el tiempo de ejecución
execution_time = end_time - start_time
print(f"Tiempo de ejecución de q3_time: {execution_time} s \n")

# Imprimir los resultados obtenidos
print("Resultados obtenidos:")
print(results, "\n")

print("Resultados humanizados:")
for i, (date, username) in enumerate(results, start=1):
    print(f"{i}. Fecha: {date}, Usuario con más publicaciones: {username}")

Tiempo de ejecución de q3_time: 10.538569927215576 s 

Resultados obtenidos:
[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)] 

Resultados humanizados:
1. Fecha: narendramodi, Usuario con más publicaciones: 2265
2. Fecha: Kisanektamorcha, Usuario con más publicaciones: 1840
3. Fecha: RakeshTikaitBKU, Usuario con más publicaciones: 1644
4. Fecha: PMOIndia, Usuario con más publicaciones: 1427
5. Fecha: RahulGandhi, Usuario con más publicaciones: 1146
6. Fecha: GretaThunberg, Usuario con más publicaciones: 1048
7. Fecha: RaviSinghKA, Usuario con más publicaciones: 1019
8. Fecha: rihanna, Usuario con más publicaciones: 986
9. Fecha: UNHumanRights, Usuario con más publicaciones: 962
10. Fecha: meenaharris, Usuario con más publicaciones: 926


### ***Conclusión*** `q3_time`


Si bien el intento definitivo que obtuvo unos mejores tiempos en ejecución es similar al intento 1, este último se descartó debido a que en varias pruebas mostró unos tiempos más elevados de lo normal.

El intento 2 no se tuvo en cuenta debido a que por la forma en la que se intento leer el JSON dejaba mucha información por fuera y su resultado final no era exacto, por ello este intento tiene los mejores tiempos de todos.

Como lo mencioné, la función final `q3_time` resultó ser una función muy corta pero eficiente para iterar y encontrar los usuarios más mencionados en base de datos.

# **R#3 - Enfoque 2:** Optimización de la memoria en uso.

### ***- Primer intento:*** función `q3_memory`

En este intento la función `q3_memory` procesa el archivo línea por línea, lo que minimiza la cantidad de datos almacenados en memoria en un momento dado. Utiliza un contador (`user_mentions`) para mantener un seguimiento eficiente del conteo de menciones de cada usuario. Y no almacena explícitamente los tweets en la memoria, lo que reduce el uso de memoria durante el procesamiento.

#### Detalles de la implementación:

1. **Inicialización del contador de menciones**:
    - Se inicializa un contador de menciones llamado `user_mentions` utilizando la clase `Counter` del módulo `collections`.

2. **Lectura del archivo JSON y procesamiento de tweets**:
    - Se abre el archivo JSON especificado en `file_path` en modo de lectura.
    - Se itera sobre cada línea del archivo utilizando un bucle `for`.
    
3. **Obtención del nombre de usuario del tweet**:
    - En cada iteración, se carga la línea como un objeto JSON utilizando `json.loads()`.
    - Se extrae el nombre de usuario del tweet accediendo a la clave `'user'` y luego a la clave `'username'`.

4. **Conteo de menciones (@) en el contenido del tweet**:
    - Se cuenta el número de menciones (@) en el contenido del tweet utilizando el método `count()` de las cadenas de Python.

5. **Actualización del contador de menciones para el usuario**:
    - Se actualiza el contador `user_mentions` para el nombre de usuario actual sumando el número de menciones contadas en el paso anterior.

6. **Obtención de los top 10 usuarios más influyentes**:
    - Una vez que se han procesado todos los tweets, se utiliza el método `most_common()` de `Counter` para obtener los 10 usuarios más influyentes con su respectivo conteo de menciones.


In [51]:
import json
from collections import Counter
from typing import List, Tuple

def q3_memory_attempt_1(file_path: str) -> List[Tuple[str, int]]:
    """
    Encuentra el top 10 histórico de usuarios más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

    Args:
    - file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
    - List[Tuple[str, int]]: Una lista de tuplas que contiene el top 10 de usuarios más influyentes, cada uno con su respectivo conteo de menciones.
    """
    # Paso 1: Inicializar un contador para almacenar el conteo de menciones de cada usuario.
    user_mentions = Counter()

    # Paso 2: Leer el archivo JSON y procesar los tweets línea por línea.
    with open(file_path, 'r') as data:
        for line in data:
            tweet = json.loads(line)

            # Paso 3: Obtener el nombre de usuario del tweet.
            username = tweet['user']['username']

            # Paso 4: Contar las menciones (@) en el contenido del tweet.
            mentions_count = tweet['content'].count('@')

            # Paso 5: Actualizar el contador de menciones para el usuario.
            user_mentions[username] += mentions_count

    # Paso 6: Obtener los top 10 usuarios más influyentes con su respectivo conteo de menciones.
    top_users = user_mentions.most_common(10)

    return top_users

### ***- Segundo intento:*** función `q3_memory`

En esta versión final, la función `q3_memory` procesa un archivo JSON que contiene tweets para encontrar los 10 usuarios más influyentes basándose en el conteo de menciones (@) que registra cada uno de ellos.

#### Detalles de la implementación:

1. **Creación del objeto Counter para registrar los usernames:**
   - Se utiliza la clase `Counter` del módulo `collections` para crear un contador que almacenará el conteo de menciones de cada usuario.

2. **Lectura de cada línea del archivo:**
   - Se utiliza un loop infinito para leer el archivo línea por línea.

3. **Carga de la línea como un objeto JSON y obtención de la lista de usuarios mencionados:**
   - Se utiliza la función `json.loads()` para cargar cada línea del archivo como un objeto JSON.
   - Se accede al atributo `'mentionedUsers'` de cada tweet para obtener la lista de usuarios mencionados en ese tweet.

4. **Verificación de si la lista de usuarios mencionados es None:**
   - Se verifica si la lista de usuarios mencionados es `None` para evitar errores al intentar acceder a los usernames.

5. **Obtención de los usernames de la lista de usuarios mencionados y actualización del contador:**
   - Se utiliza una lista de comprensión para obtener los usernames de la lista de usuarios mencionados.
   - Se utiliza el método `update()` del objeto `Counter` para actualizar el contador con los usernames obtenidos.

6. **Retorno de los 10 usernames más mencionados:**
   - Se utiliza el método `most_common(10)` del objeto `Counter` para obtener los 10 usernames más mencionados junto con su respectiva frecuencia de menciones.

Este enfoque asegura una eficiente iteración a través del archivo JSON y permite encontrar rápidamente los usuarios más mencionados en base a su frecuencia de menciones.




In [52]:
import json
from collections import Counter
from typing import List, Tuple

def q3_memory_attempt_2(file_path: str) -> List[Tuple[str, int]]:
    """
    Encuentra el top 10 histórico de usuarios más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene el top 10 de usuarios más influyentes, cada uno con su respectivo conteo de menciones.
    """
    # Paso 1: Se crea el objeto Counter para registrar los usernames
    users_counter = Counter()

    # Se abre el archivo JSON en modo de lectura
    with open(file_path, 'r') as data:
        # Loop infinito para leer el archivo línea por línea
        while True:
            # Paso 2: Lectura de cada línea del archivo
            line = data.readline()

            # Condición de salida del loop while cuando se alcanza el final del archivo
            if not line:
                break

            # Paso 3: Carga de la línea como un objeto JSON y obtención de la lista de usuarios mencionados
            mentioned_users = json.loads(line)['mentionedUsers']

            # Paso 4: Verificación de si la lista de usuarios mencionados es None
            if mentioned_users is None:
                continue

            # Paso 5: Obtención de los usernames de la lista de usuarios mencionados y actualización del contador
            usernames = [user['username'] for user in mentioned_users]
            users_counter.update(usernames)

    # Paso 6: Retorno de los 10 usernames más mencionados junto con su respectiva frecuencia de menciones
    return users_counter.most_common(10)

### ***- Intento definitivo:*** función `q3_memory`

En este intento definitivo, la función `q3_memory` procesa el archivo línea por línea, lo que minimiza la cantidad de datos almacenados en memoria en un momento dado. Se utiliza un contador (`user_counter`) en lugar de almacenar todas las listas de usuarios mencionados, lo que reduce la cantidad de memoria utilizada. Y finalmente, se utiliza un generador para extraer los nombres de usuario de cada lista de usuarios mencionados, evitando así la creación de una lista temporal.


#### Detalles de la implementación:

1. **Procesamiento del archivo línea por línea**:
    - Se inicializa un contador de usuarios `user_counter` utilizando la clase `Counter` del módulo `collections`.
    - Se abre el archivo JSON especificado en `file_path` en modo de lectura.
    - Se itera sobre cada línea del archivo utilizando un bucle `for`.
    - En cada iteración, se carga la línea como un objeto JSON utilizando `json.loads()`.
    - Se obtiene la lista de usuarios mencionados en el tweet utilizando el método `get()` para acceder a la clave `'mentionedUsers'` del objeto JSON.
    - Si hay usuarios mencionados en el tweet, se actualiza el contador de usuarios utilizando el método `update()` de `Counter`, donde se pasa un generador que extrae los nombres de usuario de la lista de usuarios mencionados.

2. **Obtención de los 10 usernames más comunes**:
    - Una vez que se han procesado todas las líneas del archivo, se devuelve una lista de tuplas que contiene los 10 nombres de usuario más comunes y su frecuencia de menciones utilizando el método `most_common()` de `Counter`.

In [53]:
import json
from collections import Counter
from typing import List, Tuple

def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función procesa un archivo JSON que contiene registros de tweets y devuelve los 10 nombres de usuario
    más mencionados en los tweets, junto con la frecuencia de menciones de cada uno.

    Args:
        file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 nombres de usuario más mencionados,
        junto con la frecuencia de menciones de cada uno.

    Raises:
        FileNotFoundError: Si el archivo especificado en file_path no se encuentra.
    """
    # Paso 1: Se procesa el archivo línea por línea
    user_counter = Counter()

    with open(file_path, 'r') as data:
        for line in data:
            tweet_data = json.loads(line)
            mentioned_users = tweet_data.get('mentionedUsers')
            if mentioned_users:
                user_counter.update(user['username'] for user in mentioned_users)

    # Paso 2: Se retornan los 10 usernames más comunes
    return user_counter.most_common(10)

## - Ejecución de la función `q3_memory`

***Medición de memoria:***

Utilizando memory_profiler para medir el uso de memoria de la función `q3_memory`. `memory_usage` devuelve una lista que contiene el uso de memoria en diferentes puntos de la ejecución de la función. En este caso, se pasa `(q3_memory, (test_file_path,))` como argumento a `memory_usage`, lo que significa que estás midiendo el uso de memoria de `q3_memory` con `test_file_path` como argumento (base de datos). Finalmente se imprime el usuo de memoria de `q3_memory`.

***2. Ejecución de q3_memory:***

Se realiza la ejecición de de `q3_memory` con `test_file_path` como argumento (base de datos) y se imprime su resultado.

***3. Impresión de resultado "Humanizado":***

Se imprime el mismo resultado anterior organizando la información para que resulta más facil de comprender.

In [54]:
import memory_profiler

# Medir el uso de memoria de q3_memory_attempt_1, con memory-profiler
mem_usage = memory_profiler.memory_usage((q3_memory_attempt_1, (test_file_path, )))

# Imprimir el uso de memoria
print(f"Uso de memoria de q3_memory_attempt_1,: {max(mem_usage)} MB \n")

# Medir el uso de memoria de q3_memory con memory-profiler
mem_usage = memory_profiler.memory_usage((q3_memory, (test_file_path, )))

# Imprimir el uso de memoria
print(f"Uso de memoria de q3_memory: {max(mem_usage)} MB \n")

# Ejecutar q3_memory y obtener los resultados
results = q3_memory(test_file_path)

# Imprimir los resultados obtenidos
print("Resultados obtenidos:")
print(results, "\n")

print("Resultados humanizados:")
for i, (date, username) in enumerate(results, start=1):
    print(f"{i}. Fecha: {date}, Usuario con más publicaciones: {username}")

Uso de memoria de q3_memory_attempt_1,: 335.60546875 MB 

Uso de memoria de q3_memory: 334.625 MB 

Resultados obtenidos:
[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)] 

Resultados humanizados:
1. Fecha: narendramodi, Usuario con más publicaciones: 2265
2. Fecha: Kisanektamorcha, Usuario con más publicaciones: 1840
3. Fecha: RakeshTikaitBKU, Usuario con más publicaciones: 1644
4. Fecha: PMOIndia, Usuario con más publicaciones: 1427
5. Fecha: RahulGandhi, Usuario con más publicaciones: 1146
6. Fecha: GretaThunberg, Usuario con más publicaciones: 1048
7. Fecha: RaviSinghKA, Usuario con más publicaciones: 1019
8. Fecha: rihanna, Usuario con más publicaciones: 986
9. Fecha: UNHumanRights, Usuario con más publicaciones: 962
10. Fecha: meenaharris, Usuario con más publicaciones: 926


### ***Conclusión*** `q3_memory`

Semejante que con q2_time, los intentos 1 y 2 se intentaron hacer utilizando el minimo de funciones adicionales, en el primero de ellos usando representación Unicode de cada emoji para hacer la búsqueda y en el segundo usando `emoji_list()`, pero al revisar su ejecución y los resultados que se obtienen no son satisfactorios, puesto que muchos Unicodes los malinterpreta y el proceso de búsqueda es muy lento.

En la versión final se utilizan `Counter`, `emoji_list()`, junto con un recorrido linea a linea del archivo para evitar la sobrecarga en memoria de información y asi, de esta manera se crea una manera simple y eficiente de hacer la búsqueda y conteo de los emojis sobre la base de datos y después de todos los intentos se obtiene en esta implementación el mejor tiempo de respuesta.



---



# **Agradecimiento 🙏**

Quiero expresar mi más sincero agradecimiento por brindarme la oportunidad de postularme al cargo de Data Engineer Challenge. Es un honor para mí tener la posibilidad de trabajar con un equipo tan talentoso y dinámico como el de LATAM. Estoy entusiasmado por la posibilidad de contribuir con mi experiencia y conocimientos, y espero poder colaborar pronto con ustedes para alcanzar metas compartidas y enfrentar desafíos emocionantes en el mundo de la ingeniería de datos. 😁

Atentamente,

Fabián Callejas
`fabiancallejas@gmail.com`